In [1]:
!pip install pandas numpy tensorflow scikit-learn


  Using cached tensorflow-2.19.0-cp311-cp311-win_amd64.whl.metadata (4.1 kB)
  Using cached keras-3.10.0-py3-none-any.whl.metadata (6.0 kB)
Using cached tensorflow-2.19.0-cp311-cp311-win_amd64.whl (375.9 MB)
Using cached keras-3.10.0-py3-none-any.whl (1.4 MB)



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: C:\Users\HADEZZZ\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [ ]:
run in terminal 

for bash:
wget http://nlp.stanford.edu/data/glove.6B.zip
unzip glove.6B.zip


for powershell:
Invoke-WebRequest -Uri "http://nlp.stanford.edu/data/glove.6B.zip" -OutFile "glove.6B.zip"

Expand-Archive -Path "glove.6B.zip" -DestinationPath "glove"

SyntaxError: invalid decimal literal (2740742414.py, line 1)

In [2]:
from datasets import load_dataset

# Load SST5 dataset
sst5_dataset = load_dataset("SetFit/sst5")
train_data = sst5_dataset["train"]
train_data[0]

C:\Users\HADEZZZ\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Repo card metadata block was not found. Setting CardData to empty.


{'text': 'a stirring , funny and finally transporting re-imagining of beauty and the beast and 1930s horror films',
 'label': 4,
 'label_text': 'very positive'}

In [3]:
twitter_dataset = load_dataset("tweet_eval", "sentiment")
twt_train_data = twitter_dataset["train"]
twt_train_data[0]

{'text': '"QT @user In the original draft of the 7th book, Remus Lupin survived the Battle of Hogwarts. #HappyBirthdayRemusLupin"',
 'label': 2}

In [5]:
from datasets import DatasetDict, Value

def convert_twitter(example):
    return {"text": example["text"], "label": int(example["label"])}


twitter_mapped = twitter_dataset.map(convert_twitter)
print(twitter_mapped["train"][0])


{'text': '"QT @user In the original draft of the 7th book, Remus Lupin survived the Battle of Hogwarts. #HappyBirthdayRemusLupin"', 'label': 2}


In [7]:
def convert_sst5(example):
    label = int(example["label"])
    # label = example["label"]
    if label in [0, 1]:
        return {"text": example["text"], "label": 0}
    elif label == 2:
        return {"text": example["text"], "label": 1}
    else:
        return {"text": example["text"], "label": 2}

# sst5_mapped = sst5_dataset.map(convert_sst5)
sst5_mapped = sst5_dataset.map(convert_sst5, remove_columns=["label_text"])
print(sst5_mapped["train"][0])
print(sst5_dataset["train"][0])

{'text': 'a stirring , funny and finally transporting re-imagining of beauty and the beast and 1930s horror films', 'label': 2}
{'text': 'a stirring , funny and finally transporting re-imagining of beauty and the beast and 1930s horror films', 'label': 4, 'label_text': 'very positive'}


In [8]:
from datasets import concatenate_datasets, DatasetDict
from datasets import ClassLabel

# Define common ClassLabel
label_class = ClassLabel(names=["negative", "neutral", "positive"])

# Ensure label columns are the same type before combining
sst5_mapped = sst5_mapped.cast_column("label", label_class)
twitter_mapped = twitter_mapped.cast_column("label", label_class)

full_combined_dataset = concatenate_datasets([
    sst5_mapped["train"], 
    sst5_mapped["test"], 
    twitter_mapped["train"], 
    twitter_mapped["test"],
    sst5_mapped["validation"], 
    twitter_mapped["validation"]
    
])

# Shuffled
full_combined_dataset = full_combined_dataset.shuffle(seed=42)
print(full_combined_dataset)


Dataset({
    features: ['text', 'label'],
    num_rows: 71754
})


In [9]:
combined_dataset = full_combined_dataset.train_test_split(
    test_size=0.2,
    stratify_by_column="label"  # makes sure all labels are balanced
)

combined_train = combined_dataset["train"]
combined_test = combined_dataset["test"]

In [10]:
import re
import string

def clean_text(combined_train):
    combined_train = combined_train.lower()
    combined_train = re.sub(r"http\S+|www\S+|https\S+", '', combined_train, flags=re.MULTILINE)  # Remove URLs
    combined_train = re.sub(r'\@\w+|\#', '', combined_train)  # Remove @mentions and hashtags
    combined_train = combined_train.translate(str.maketrans('', '', string.punctuation))  # Remove punctuation
    combined_train = re.sub(r'\d+', '', combined_train)  # Remove digits
    combined_train = re.sub(r'\s+', ' ', combined_train).strip()  # Remove extra whitespaces
    return combined_train
full_combined_dataset = full_combined_dataset.map(lambda x: {"text": clean_text(x["text"])})
# combined_train = combined_train.map(lambda x: {"text": clean_text(x["text"])})
# combined_test = combined_test.map(lambda x: {"text": clean_text(x["text"])})

In [9]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Load dataset
df = full_combined_dataset.to_pandas()  # Convert to pandas DataFrame
texts = df['text'].values  # assuming the column is named 'text'
labels = df['label'].values  # assuming the column is named 'label'
print(texts[0], labels[0])

# Encode labels (positive, neutral, negative)
le = LabelEncoder()
labels_encoded = le.fit_transform(labels)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(texts, labels_encoded, test_size=0.2, random_state=42)

# Tokenize text
max_words = 20000
max_len = 100

tokenizer = Tokenizer(num_words=max_words, oov_token="<OOV>")
tokenizer.fit_on_texts(X_train)

X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

X_train_pad = pad_sequences(X_train_seq, maxlen=max_len)
X_test_pad = pad_sequences(X_test_seq, maxlen=max_len)


charlotte the new day triple h john cena and others sit down with michael cole every wednesday in 1


In [10]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout

model = Sequential([
    Embedding(input_dim=max_words, output_dim=128, input_length=max_len),
    LSTM(128, return_sequences=False),
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(3, activation='softmax')  # 3 classes: positive, neutral, negative
])

model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.summary()


C:\Users\HADEZZZ\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\core\embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [11]:
history = model.fit(X_train_pad, y_train,
                    validation_split=0.1,
                    epochs=10,
                    batch_size=64,
                    verbose=1)


Epoch 1/10
808/808 ━━━━━━━━━━━━━━━━━━━━ 66s 80ms/step - accuracy: 0.5101 - loss: 0.9564 - val_accuracy: 0.6522 - val_loss: 0.7604
Epoch 2/10
808/808 ━━━━━━━━━━━━━━━━━━━━ 48s 59ms/step - accuracy: 0.7194 - loss: 0.6526 - val_accuracy: 0.6553 - val_loss: 0.7571
Epoch 3/10
808/808 ━━━━━━━━━━━━━━━━━━━━ 92s 114ms/step - accuracy: 0.7831 - loss: 0.5130 - val_accuracy: 0.6408 - val_loss: 0.8366
Epoch 4/10
808/808 ━━━━━━━━━━━━━━━━━━━━ 113s 140ms/step - accuracy: 0.8345 - loss: 0.3974 - val_accuracy: 0.6335 - val_loss: 1.0245
Epoch 5/10
808/808 ━━━━━━━━━━━━━━━━━━━━ 122s 150ms/step - accuracy: 0.8761 - loss: 0.3046 - val_accuracy: 0.6161 - val_loss: 1.2314
Epoch 6/10
808/808 ━━━━━━━━━━━━━━━━━━━━ 103s 127ms/step - accuracy: 0.9036 - loss: 0.2378 - val_accuracy: 0.6196 - val_loss: 1.5289
Epoch 7/10
808/808 ━━━━━━━━━━━━━━━━━━━━ 106s 131ms/step - accuracy: 0.9237 - loss: 0.1918 - val_accuracy: 0.6156 - val_loss: 1.7621
Epoch 8/10
808/808 ━━━━━━━━━━━━━━━━━━━━ 108s 133ms/step - accuracy: 0.9416 - loss

In [12]:
loss, accuracy = model.evaluate(X_test_pad, y_test, verbose=0)
print(f"Test Accuracy: {accuracy * 100:.2f}%")


Test Accuracy: 59.07%


with glove and bidirectional

In [22]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Load your data
df = full_combined_dataset.to_pandas()
texts = df['text'].values
labels = df['label'].values

# Encode labels
le = LabelEncoder()
labels = le.fit_transform(labels)

# Split data
X_train, X_test, y_train, y_test = train_test_split(texts, labels, test_size=0.2, random_state=42)

# Tokenization
max_words = 20000
max_len = 100

tokenizer = Tokenizer(num_words=max_words, oov_token="<OOV>")
tokenizer.fit_on_texts(X_train)
word_index = tokenizer.word_index

X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

X_train_pad = pad_sequences(X_train_seq, maxlen=max_len)
X_test_pad = pad_sequences(X_test_seq, maxlen=max_len)

# Load GloVe embeddings
embedding_index = {}
with open("D:\\shadaab project\\Model\\glove\\glove.6B.100d.txt", encoding="utf8") as f:
    for line in f:
        values = line.split()
        word = values[0]
        vector = np.asarray(values[1:], dtype='float32')
        embedding_index[word] = vector

# Build embedding matrix
embedding_dim = 100
embedding_matrix = np.zeros((max_words, embedding_dim))
for word, i in tokenizer.word_index.items():
    if i < max_words and word in embedding_index:
        embedding_matrix[i] = embedding_index[word]


In [33]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense, Dropout

model = Sequential([
    Embedding(input_dim=max_words,
              output_dim=embedding_dim,
              weights=[embedding_matrix],
              input_length=max_len,
              trainable=True),  # freeze GloVe weights
    Bidirectional(LSTM(64, return_sequences=True, dropout=0.3, recurrent_dropout=0.2)),
    Bidirectional(LSTM(64, dropout=0.2, recurrent_dropout=0.2)),
    # Dropout(0.5),
    Dense(64, activation=tf.nn.gelu),
    Dropout(0.4),
    Dense(3, activation='softmax')  # 3 sentiment classes
])

model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.summary()


Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_4 (Embedding)         │ ?                      │     2,000,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_5 (Bidirectional) │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_6 (Bidirectional) │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,000,000 (7.63 MB)

 Trainable params: 2,000,000 (7.63 MB)

 Non-trainable params: 0 (0.00 B)

In [34]:
from tensorflow.keras.callbacks import ReduceLROnPlateau

lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, verbose=1)


In [35]:
from tensorflow.keras.callbacks import EarlyStopping

early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)


In [36]:
history = model.fit(X_train_pad, y_train,
                    validation_data=(X_test_pad, y_test),
                    # validation_split=0.1,
                    epochs=20,
                    batch_size=64,
                    callbacks=[early_stop, lr_scheduler],
                    verbose=1)


Epoch 1/20
897/897 ━━━━━━━━━━━━━━━━━━━━ 264s 284ms/step - accuracy: 0.5285 - loss: 0.9429 - val_accuracy: 0.6528 - val_loss: 0.7620 - learning_rate: 0.0010
Epoch 2/20
897/897 ━━━━━━━━━━━━━━━━━━━━ 267s 298ms/step - accuracy: 0.6723 - loss: 0.7347 - val_accuracy: 0.6685 - val_loss: 0.7325 - learning_rate: 0.0010
Epoch 3/20
897/897 ━━━━━━━━━━━━━━━━━━━━ 352s 331ms/step - accuracy: 0.7129 - loss: 0.6578 - val_accuracy: 0.6700 - val_loss: 0.7344 - learning_rate: 0.0010
Epoch 4/20
897/897 ━━━━━━━━━━━━━━━━━━━━ 0s 292ms/step - accuracy: 0.7451 - loss: 0.5978
Epoch 4: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
897/897 ━━━━━━━━━━━━━━━━━━━━ 278s 310ms/step - accuracy: 0.7451 - loss: 0.5978 - val_accuracy: 0.6704 - val_loss: 0.7452 - learning_rate: 0.0010
Epoch 5/20
897/897 ━━━━━━━━━━━━━━━━━━━━ 275s 306ms/step - accuracy: 0.7724 - loss: 0.5347 - val_accuracy: 0.6640 - val_loss: 0.7816 - learning_rate: 5.0000e-04
Epoch 6/20
897/897 ━━━━━━━━━━━━━━━━━━━━ 0s 291ms/step - accurac

In [37]:
loss, accuracy = model.evaluate(X_test_pad, y_test, verbose=0)
print(f"Test Accuracy: {accuracy * 100:.2f}%")


Test Accuracy: 66.85%


In [39]:
!pip install tf-keras

   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
   ---------------------------------------- 1.7/1.7 MB 15.6 MB/s eta 0:00:00



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: C:\Users\HADEZZZ\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [45]:
import os
os.environ["TF_KERAS"] = "1" 

In [49]:
import numpy as np
import pandas as pd
import tensorflow as tf
import transformers
from tensorflow import keras
from transformers import BertTokenizer, TFBertModel
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Bidirectional, LSTM, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.layers import Layer

print("Transformers:", transformers.__version__)
print("TensorFlow:", tf.__version__)
print("Keras:", keras.__version__)


df = full_combined_dataset.to_pandas()  # Replace with your actual CSV

# Encode labels (e.g., 'positive', 'negative', 'neutral' -> 0, 1, 2)
label_encoder = LabelEncoder()
df['label'] = label_encoder.fit_transform(df['label'])

texts = df['text'].tolist()
labels = df['label'].tolist()


max_len = 128  # max token length
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

encodings = tokenizer(
    texts,
    truncation=True,
     padding='max_length',
    max_length=max_len,
    return_tensors='tf'
)

input_ids = encodings['input_ids'].numpy()
attention_mask = encodings['attention_mask'].numpy()
labels = tf.convert_to_tensor(labels).numpy()

# Train/test split
train_ids, val_ids, train_mask, val_mask, train_labels, val_labels = train_test_split(
    input_ids, attention_mask, labels, test_size=0.2, random_state=42
)

# ===============================
# 3. Build BERT + LSTM Model
# ===============================
bert_model = TFBertModel.from_pretrained('bert-base-uncased')

input_ids_layer = Input(shape=(max_len,), dtype=tf.int32, name='input_ids')
mask_layer = Input(shape=(max_len,), dtype=tf.int32, name='attention_mask')





class BertLayer(Layer):
    def __init__(self, **kwargs):
        super(BertLayer, self).__init__(**kwargs)
        self.bert = TFBertModel.from_pretrained('bert-base-uncased')

    def call(self, inputs):
        return self.bert(inputs).last_hidden_state

bert_outputs = BertLayer()({
    'input_ids': input_ids_layer,
    'attention_mask': mask_layer
})

# BERT outputs
# bert_outputs = bert_model({
#     'input_ids': input_ids_layer,
#     'attention_mask': mask_layer
# }).last_hidden_state


# LSTM layer
x = Bidirectional(LSTM(64, return_sequences=False))(bert_outputs)

# Dense layers
x = Dense(64, activation=tf.nn.gelu)(x)
x = Dropout(0.3)(x)
output = Dense(3, activation='softmax')(x)  # 3 sentiment classes

# Define model
model = Model(inputs=[input_ids_layer, mask_layer], outputs=output)

# Compile model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.summary()

# ===============================
# 4. Train Model
# ===============================
history = model.fit(
    x={'input_ids': train_ids, 'attention_mask': train_mask},
    y=train_labels,
    validation_data=({'input_ids': val_ids, 'attention_mask': val_mask}, val_labels),
    batch_size=16,
    epochs=4
)


Transformers: 4.51.0
TensorFlow: 2.19.0
Keras: 3.10.0


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

Model: "functional_6"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ attention_mask      │ (None, 128)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_ids           │ (None, 128)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bert_layer_1        │ (None, 128, 768)  │          0 │ attention_mask[0… │
│ (BertLayer)         │                   │            │ input_ids[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bidirectional_8     │ (None, 128)       │    426,496 │ bert_layer_1[0][… │
│ (Bidirectional)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_12 (Dense)    │ (None, 64)        │      8,256 │ bidirectional_8[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_8 (Dropout) │ (None, 64)        │          0 │ dense_12[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_13 (Dense)    │ (None, 3)         │        195 │ dropout_8[0][0]   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 434,947 (1.66 MB)

 Trainable params: 434,947 (1.66 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/4
 128/3588 ━━━━━━━━━━━━━━━━━━━━ 2:17:35 2s/step - accuracy: 0.4755 - loss: 1.0129

KeyboardInterrupt: 